# 経済産業省の通商白書を全文検索するAPIサーバの計画

Date: 2024/04/29

## 前処理
- Beautiful SoapでPDFファイルのリンクリスト作成
- PyMuPDFで各PDFファイルのText抽出しSQLiteへ格納
- 同時にspaCyでNERを行い、その結果をSQLiteへ格納

```
Table "source" <= 経産省、総務省、XX株式会社といった白書やIR資料を提供している団体名
source, base_url

Table "links" <= Beautiful Soapで抽出したPDFファイルのURl
id(sha1(url)), url, title, fk(source:source)

Table "texts" <= PyMuPDFで抽出したテキスト
fk(info_files:id), page, text

Table "named_entities" <= spaCyで抽出したNamed Entity
fk(info_files:id), page, named_entity, label
```

## APIサーバ

#### /search?keywords=\<keywords\>

{id: \<id\>, filename: \<filename\>, page: \<page\>, source: \<source\>}

#### /fetch_pdf/\<id\>?\<keywords\>

該当キーワードをハイライトしてPDFを返送。keywordsなしのときはハイライトせず返信。




## Table: source

In [67]:
import sqlite3

source_meti = "経済産業省 通商白書"
base_url_meti = "https://www.meti.go.jp/report/tsuhaku2023"

with sqlite3.connect('database/search.db') as conn:
    cur = conn.cursor()
    cur.execute('CREATE TABLE IF NOT EXISTS search (source TEXT PRIMARY KEY, base_url TEXT)')
    cur.execute('DELETE FROM search')
    cur.execute(f'INSERT INTO search (source, base_url) VALUES ("{source_meti}", "{base_url_meti}")')

## Table: links

PDFリンク抽出

In [3]:
url_meti = "https://www.meti.go.jp/report/tsuhaku2023/whitepaper_2023.html"

In [4]:
import requests
from bs4 import BeautifulSoup

In [14]:
resp = requests.get(url_meti)
html_doc = resp.content.decode('utf-8')

In [15]:
soup = BeautifulSoup(html_doc, 'html.parser')

In [61]:
all_ul = soup.find_all("ul", {"class": "lnkLst"})
all_as = []
for ul in all_uls:
    all_as.extend(ul.find_all('a'))

# [[url, title], ...]
links = filter(lambda x: x[0].endswith('.pdf') ,[[a['href'], a.text] for a in all_as])